# Lumped Parameter Model 

Adaptado de Xiao et al (2013) Lumped parameter model for charge–discharge cycle of adsorptive
hydrogen storage system

Autor: Kieran Conradie


Las ecuaciones que gobiernan el sistema propuesto en el artículo son: 

Masa:
$$ \frac{dm_t}{dt}=\dot{m_i}-\dot{m_e}$$
Las cual puede ser separada en dos casos:
    
1. Carga: $\frac{dm_t}{dt}=\dot{m_i}$
2. Descarga: $\frac{dm_t}{dt}=-\dot{m_e}$ 

Dentro de las ecuaciones de masa existen otras que ayudan a especificar el problema: 
$$m_t = m_g + m_a$$
$$m_a = n_a M_{H_2} m_s$$

Donde los moles de hidrógeno adsorbido están dados por:
$$n_a = n_0 \exp\left[-\left(\frac{RT}{\alpha + \beta T}\right)^b \ln{\left(\frac{p_0}{p}\right)}^b\right]$$
y la presión esta dada por:
$$p=\frac{(m-m_a)ZRT}{M_{H_2}V\epsilon_b}$$

Temperatura:
$$\frac{dT}{dt}=\frac{\dot{m_i}h_i - \dot{m_e}h_e + \frac{dm_a}{dt}\frac{\Delta{H}}{M_{H_2}}-h_fA_e(T-T_f)}{m_sc_s+m_ac_{va}+m_gc_{vg}+m_wc_w}$$

Además se debe notar que en Xiao et al. (2012) Finite element simulation of heat and mass transfer in activated carbon hydrogen storage tank, se obtuvo:
$$\frac{\partial{n_a}}{\partial t}=mn_a\left(\frac{RT}{\alpha + \beta T}\ln{\left(\frac{p_0}{p}\right)}\right)^b \left[\frac{1}{\ln{(p_0/p)}}\frac{1}{p}\frac{\partial p}{\partial t}-\frac{\alpha}{\alpha + \beta T}\frac{1}{T}\frac{\partial T}{\partial t}\right]$$

In [1]:
# Integración numérica de sistemas de ecuaciones diferenciales ordinarias
from scipy.integrate import solve_ivp

# Minimización de funciones objetivos para ajuste de parámetros
from scipy.optimize import least_squares
import numpy as np
import matplotlib.pyplot as plt

### Definición de la función con el sistema de ecuaciones diferenciales

In [2]:
# Por el momento la función solo trabajara para el proceso de descarga, después se le podria dar un parametro mas a la función 
# que use un kwarg del tipo proceso = carga/descarga y en base a eso realizar cálculos
def adsorcion_hidrogeno(t, y, c_s, c_p, c_w, m_s, m_w, M_H2, R, alpha, beta, epsilon, volumen, area, p_0, n_0, b, m_dot, H, h_f, p_i, T_f):
    
    # Desempacar variables
    n_a = y[0] # Isoterma de adsorcion 
    T = y[1] # Temperatura 
    p = y[2]
    
    
    # Inicializar variables como las masas (m_t, m_a, m_g) y la presión (Valor dado por las condiciones iniciales)
    m_t = 0 # Al principio del experimento el contenedor comienza vacio
    n_a = 0 # Al empezar vacio la masa adsorbida también debe ser 0, quizas se pueda usar para un tiempo "Justo" después de t = 0 s
    m_a = M_H2*m_s*n_a
    m_g = m_t - m_a # Relación dada en el artículo

    
    # Calcular cantidades dependientes de las variables independientes
    n_a = n_0 * np.exp(-((R*T)/(alpha + beta*T))**b * np.log(p_0/p)**b)
    dH = alpha * (np.log(p_0/p))**(1/b)
    """ Falta implementar la función para la presión y las relaciones entre masa en fase gas, masa adsorbida y presión
    dada por la ecuaciones del artículo """
    """ Ahora que se esta usando p como una variable del vector y no estoy seguro si se debe implementar la función
    para p."""
    
    # Como todas las ecuaciones dependen una de la otra se declara un valor para dT_dt y dp_dt inicial
    dT_dt = 0
    dp_dt = 0
    
    # Ecuación diferencial para la isoterma
    dn_dt = (m_t*n_a*(((R*T)/(alpha+beta*T))*np.log(p_0/p))**b * 
             ((1/np.log(p_0/p))*(1/p)*dp_dt - (alpha/(alpha+beta*T))*(1/T)*dT_dt))
    
    # Ecuación diferencial para la temperatura
    # Por el momento asumir que c_v = c_p para el hidrogeo (Solo por flojera, después se debe usar una correlación)
    dT_dt = (m_dot*H + M_H2*m_s*dn_dt - h_f*area*(T-T_f))/(m_s*c_s + m_a*c_p + m_g*c_p + m_w*c_w)
    
    # Ecuación diferencial para la presión
    dp_dt = 0 #Placeholder, falta hacer la derivada para una forma explicita
    
    # Empacar el vector del lado derecho en un vector 2x1
    dy = np.array([dn_dt, dT_dt, dp_dt])
    
    return dy

### Parametros y condiciones

In [3]:
# Parametros

# Calores especificos
c_s = 825 # Calor especifico del carbón activado (J kg-1 K-1)
c_p = 10167 # Calor especifico del hidrogeno (J kg-1 K-1)
c_w = 468 # Calor especifico paredes de acero (J kg-1 K-1)

# Masas
m_s = 0.671 # masa carbón activado (kg)
m_w = 3.714 # Masa paredes de acero (kg)
M_H2 = 2.0159E-3 # Masa molar del hidregeno (kg mol-1)

# Constantes 
R = 8.314 # J mol-1 K-1
alpha = 3080 # Factor entalpico (J mol-1)
beta = 18.9 # Factor entropico (J mol-1 K-1)
epsilon_b = 0.49 
b = 2

# Dimensiones estanque
V = 2.4946 # Volumen de estanque (L)
A_e = 0.1277 # Área superficial estanque

# Otros
p_0 = 1470 # Presion de saturacion (MPa)
n_0 = 71.6 # Cantidad limite de adsorcion (mol kg-1)

In [4]:
# Condiciones iniciales y de borde
p_i = [0.033, 0.033, 0.033, 0.032, 0.049, 0.032] #MPa
T_i = [281, 282, 280.2, 301.5, 302, 302.4] #K 
T_f = [282.5, 284.5, 282.2, 301.7, 302.5, 302.5] #K
h_f = [36, 36, 36, 36, 36, 36]
print(f'Las presiones inciales son: {p_i}\nLas Temperaturas inciales son: {T_i}\nLas temperaturas finales son: {T_f}\nh_f es: {h_f}')

Las presiones inciales son: [0.033, 0.033, 0.033, 0.032, 0.049, 0.032]
Las Temperaturas inciales son: [281, 282, 280.2, 301.5, 302, 302.4]
Las temperaturas finales son: [282.5, 284.5, 282.2, 301.7, 302.5, 302.5]
h_f es: [36, 36, 36, 36, 36, 36]


### Modelo de adsorcion

In [5]:
def modified_dubinin_astakhov(n_0, R, T, alpha, beta, b, p_0, p):
    
    return n_a

def calor_isoterico_adsorcion(alpha, n_0, n, b):
    delH = alpha * (np.log(n_0/n)**(1/b))
    return delH

### To-Do
#### Forma de resolución al sistema
----

#### Planificación con carta Gantt (Para hoy)